In [3]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125633")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-125633
Azure region: southcentralus
Subscription id: 422c78cb-7b9f-41e7-a94c-e144f7afde5e
Resource group: aml-quickstarts-125633


In [4]:
import os
#files and directories
cc=os.getcwd
os.listdir(os.curdir)  

['.ipynb_aml_checkpoints', 'outputs', 'train.py', 'udacity-project.ipynb']

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_cluster_name = "compute-target"

# Verify that cluster does not exist already
try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice

# Specify parameter sampler
### YOUR CODE HERE ###
ps = RandomParameterSampling({"--C": uniform(0.05, 2), "--max_iter": choice(range(10,200,20)) })

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, slack_amount=None, delay_evaluation=0)

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
est = SKLearn(source_directory =".", compute_target=compute_cluster, entry_script="train.py")


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = HyperDriveConfig(
    hyperparameter_sampling = ps, 
    primary_metric_name ='Accuracy', 
    primary_metric_goal = PrimaryMetricGoal.MAXIMIZE, 
    max_total_runs = 12, 
    max_concurrent_runs=4, 
    policy=policy, 
    estimator=est
)

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [12]:
best_run=hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
best_run_files = best_run.get_file_names()

print('Bets Run ID', best_run.id)
print('\n Accuracy', best_run_metrics['Accuracy'])

Bets Run ID HD_e55958f8-e1a2-460e-8346-a6b30e5f03ed_0

 Accuracy 0.9072837632776934


In [10]:
import joblib
# Get your best run and save the model from that run.

best_model = best_run.register_model(model_name='model_log_hd', model_path='outputs/model_hd.joblib')

In [16]:
best_run.get_details()['runDefinition']['arguments']

['--C', '0.5406580442529956', '--max_iter', '130']

In [17]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path_to_csv = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'

ds = TabularDatasetFactory.from_delimited_files(path=path_to_csv)

In [22]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [23]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown
26465,52,1,0,1,0,8,3,13,14,999,...,1,0,0,0,0,1,0,0,0,0
1387,41,1,0,1,0,6,4,84,3,999,...,0,1,1,0,0,0,0,0,0,0
22628,27,0,0,0,0,7,1,331,2,999,...,1,0,0,0,0,0,0,1,0,0
8336,58,1,0,1,0,6,2,137,3,999,...,0,1,1,0,0,0,0,0,0,0
19898,36,0,0,1,0,5,1,143,4,999,...,0,1,1,0,0,0,0,0,0,0


In [24]:
x_train.shape

(26360, 39)

In [25]:
x_train['y']=y_train
x_test['y']=y_test


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [26]:
x_train.shape

(26360, 40)

In [27]:
x_train.head()

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
26465,52,1,0,1,0,8,3,13,14,999,...,0,0,0,0,1,0,0,0,0,0
1387,41,1,0,1,0,6,4,84,3,999,...,1,1,0,0,0,0,0,0,0,0
22628,27,0,0,0,0,7,1,331,2,999,...,0,0,0,0,0,0,1,0,0,0
8336,58,1,0,1,0,6,2,137,3,999,...,1,1,0,0,0,0,0,0,0,0
19898,36,0,0,1,0,5,1,143,4,999,...,1,1,0,0,0,0,0,0,0,0


In [28]:
y_train.head()

26465    0
1387     0
22628    0
8336     0
19898    0
Name: y, dtype: int64

In [29]:
columns=x_train.columns
x_train.to_csv(path_or_buf='trainset.csv', columns=columns, header=True, index=False) 
x_test.to_csv(path_or_buf='testset.csv', columns=columns, header=True, index=False) 

In [30]:
from azureml.data import TabularDataset 
datastore=ws.get_default_datastore() 
datastore.upload_files(['trainset.csv'])

Uploading an estimated of 1 files
Uploading trainset.csv
Uploaded trainset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [31]:
train=TabularDatasetFactory.from_delimited_files([(datastore, 'trainset.csv')])

datastore.upload_files(['testset.csv'])

Uploading an estimated of 1 files
Uploading testset.csv
Uploaded testset.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [32]:
test=TabularDatasetFactory.from_delimited_files([(datastore, 'testset.csv')])

In [35]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_setting={
    "featurization": "auto",
    "experiment_timeout_minutes": 30,
    "enable_early_stopping": True,
    "verbosity": logging.INFO,
    "compute_target": compute_cluster
}

task="classification" 
automl_config = AutoMLConfig( 
    task=task, 
    primary_metric='accuracy', 
    training_data=train, 
    validation_data = test, 
    label_column_name='y', 
    **automl_setting
)

In [36]:
# Submit your automl run

automl_run = exp.submit(automl_config, show_output=True)
RunDetails(automl_run).show()

Running on remote.
Running on remote compute: compute-target
Parent Run ID: AutoML_736adb3b-75b7-4916-bef1-ff1f3c9b6b0c

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of t

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [38]:
# Retrieve and save your best automl model.
automl_run, fitted_automl_model = automl_run.get_output() 
print(fitted_automl_model) #To print
import joblib  
joblib.dump(fitted_automl_model, "fitted_automl_model.joblib")

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(allow_chargram=None, enable_dnn=None,
                                 enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedso...
                                                                                                  loss='log',
                                                                                                  max_iter=1000,
                                                                                                  n_jobs=1,
                           

['fitted_automl_model.joblib']

In [39]:
automl_run_metrics = automl_run.get_metrics()
automl_run_metrics

{'accuracy': 0.9180576631259484,
 'balanced_accuracy': 0.7717510104152895,
 'f1_score_weighted': 0.9160528654252161,
 'matthews_correlation': 0.57079064952433,
 'log_loss': 0.20360863460153603,
 'precision_score_weighted': 0.9145508353358254,
 'recall_score_weighted': 0.9180576631259484,
 'recall_score_macro': 0.7717510104152895,
 'norm_macro_recall': 0.543502020830579,
 'recall_score_micro': 0.9180576631259484,
 'f1_score_macro': 0.7847643185618729,
 'AUC_weighted': 0.9466623935853943,
 'weighted_accuracy': 0.9545427685859234,
 'AUC_macro': 0.9466623935853943,
 'f1_score_micro': 0.9180576631259484,
 'average_precision_score_weighted': 0.9545214827713775,
 'average_precision_score_micro': 0.9809451729505554,
 'precision_score_macro': 0.799724704874617,
 'average_precision_score_macro': 0.8217823729839481,
 'precision_score_micro': 0.9180576631259484,
 'AUC_micro': 0.9801218565859431,
 'accuracy_table': 'aml://artifactId/ExperimentRun/dcid.AutoML_736adb3b-75b7-4916-bef1-ff1f3c9b6b0c_30/

In [40]:
print('Bets Run ID', automl_run.id)
print('\n Accuracy', automl_run_metrics['accuracy'])

Bets Run ID AutoML_736adb3b-75b7-4916-bef1-ff1f3c9b6b0c_30

 Accuracy 0.9180576631259484


In [43]:
fitted_automl_model.named_steps['datatransformer'].get_featurization_summary()

[{'RawFeatureName': 'age',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'marital',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'default',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'housing',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'loan',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'month',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'day_of_week',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
 

In [57]:
automl_run.get_details()

{'runId': 'AutoML_736adb3b-75b7-4916-bef1-ff1f3c9b6b0c_30',
 'target': 'compute-target',
 'status': 'Completed',
 'startTimeUtc': '2020-11-11T09:45:14.010325Z',
 'endTimeUtc': '2020-11-11T09:45:59.326497Z',
 'properties': {'runTemplate': 'automl_child',
  'pipeline_id': '__AutoML_Ensemble__',
  'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'udacity-project\',\'compute_target\':\'compute-target\',\'subscription_id\':\'422c78cb-7b9f-41e7-a94c-e144f7afde5e\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_736adb3b-75b7-4916-bef1-ff1f3c9b6b0c_30","experiment_name":"udacity-project","workspace_name":"quick-starts-ws-125633","subscription_id":"422c78cb-7b9f-41e

In [59]:
compute_cluster.delete()

ComputeTargetException: ComputeTargetException:
	Message: Received bad response from Resource Provider:
Response Code: 400
Headers: {'Cache-Control': 'no-cache', 'Pragma': 'no-cache', 'Content-Length': '268', 'Content-Type': 'application/json; charset=utf-8', 'Expires': '-1', 'x-ms-correlation-request-id': '13c44631-2c62-4103-a636-fbf50fdd7247', 'x-ms-ratelimit-remaining-subscription-deletes': '14998', 'Request-Context': 'appId=cid-v1:2d2e8e63-272e-4b3c-8598-4ee570a0e70d', 'x-ms-response-type': 'standard', 'x-ms-request-id': '|00-5b00e03d85fdea4cab1fb566cd0bb182-6f62f744a4aacf40-00.29026be5_', 'X-Content-Type-Options': 'nosniff', 'x-request-time': '0.336', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'x-ms-routing-request-id': 'SOUTHCENTRALUS:20201111T101139Z:13c44631-2c62-4103-a636-fbf50fdd7247', 'Date': 'Wed, 11 Nov 2020 10:11:38 GMT'}
Content: b'{"error":{"code":"ResourceNotReady","message":"Compute with name compute-target is already being deleted.","innererr